quantum LDPCcode [[18,4,4]]

In [1]:
import numpy as np
from mip import Model, xsum, minimize, BINARY
from mip import OptimizationStatus
from bposd.css import css_code
from ldpc import bposd_decoder
import pickle
import itertools 
from scipy.sparse import coo_matrix, hstack 
from ldpc import mod2
from tabulate import tabulate
import matplotlib.pyplot as plt

In [2]:
import sys
import os
sys.path.append(os.path.abspath("..")) 

from functions_BB_code import ell, m, n, n2, A, B, AT, BT, num_end 
from functions_BB_code import distance_test, rank2, get_connection_Tanner, get_layout
from functions_BB_code import get_SM_circuit_parallel, get_SM_circuit_separate
from functions_BB_code import simulate_circuit_Lx, simulate_circuit_Lz
from functions_BB_code import get_Set_noisy_circuits_logical_X, get_Set_noisy_circuits_logical_Z
from functions_BB_code import decoding_X_matrix, decoding_Z_matrix
from functions_BB_code import simulate_syndrome_Lx, simulate_syndrome_Lz 
from functions_BB_code import generate_noisy_circuit
# from functions_BB_code import error_decoding

## Code construction

In [3]:
# [[18, 4, 4]]
ell,m = 3, 3
a1,a2,a3 = 1, 0, 2
b1,b2,b3 = 1, 0, 2

# code length
n = 2*m*ell;  n2 = m*ell

I_ell = np.identity(ell,dtype=int) ; I_m = np.identity(m,dtype=int); I = np.identity(ell*m,dtype=int)
x = {} ;  y = {}

for i in range(ell):
	x[i] = np.kron(np.roll(I_ell,i,axis=1),I_m)
for i in range(m):
	y[i] = np.kron(I_ell,np.roll(I_m,i,axis=1))

A = (x[a1%ell] + y[a2%m] + y[a3%m]) % 2
B = (y[b1%m] + x[b2%ell] + x[b3%ell]) % 2

A1 = x[a1%ell]; A2 = y[a2%m]; A3 = y[a3%m]
B1 = y[b1%m]; B2 = x[b2%ell]; B3 = x[b3%ell]

AT = np.transpose(A) ; BT = np.transpose(B)

# Testing CSS code
hx = np.hstack((A,B));  hz = np.hstack((BT,AT))

remove_X_list = [2,8] ;  remove_Z_list = [3,4] ;

hx = np.delete(hx, remove_X_list, axis=0) ; hz = np.delete(hz, remove_Z_list, axis=0)

# number of logical qubits
k = n - rank2(hx) - rank2(hz)

qcode=css_code(hx=hx, hz=hz)
# qcode.compute_code_distance()  # this will take some time!
print('Testing CSS code...')
qcode.test()

Testing CSS code...
<Unnamed CSS code>, (3,6)-[[18,4,nan]]
 -Block dimensions: Pass
 -PCMs commute hz@hx.T==0: Pass
 -PCMs commute hx@hz.T==0: Pass
 -lx \in ker{hz} AND lz \in ker{hx}: Pass
 -lx and lz anticommute: Pass
 -<Unnamed CSS code> is a valid CSS code w/ params (3,6)-[[18,4,nan]]


True

In [4]:
# logical operator
lz = qcode.lz ;  lx = qcode.lx
# we choose a set of basis such that each logical X anticommute with its corresponding logical Z.
lz[1] = (lz[1]+lz[2]) %2 ; lz[3] = (lz[3]+lz[0]) %2

lz_copy = lz.copy()
lz_copy[0] = lz[1] ;  lz_copy[1] = lz[0] ;  lz_copy[2] = lz[3] ;  lz_copy[3] = lz[2] ;
lz = lz_copy
(lx@lz.T) %2

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]])

In [5]:
# Distance_test
print('Computing code distance...')
d = n
for i in range(k):
	w = distance_test(hz,lz[i,:])
	print('Logical qubit=',i,'Distance=',w)
	d = min(d,w)
    
for i in range(k):
	w = distance_test(hx,lx[i,:])
	print('Logical qubit=',i,'Distance=',w)
	d = min(d,w)
print('Code parameters: n,k,d=',n,k,d)

Computing code distance...
Logical qubit= 0 Distance= 4
Logical qubit= 1 Distance= 4
Logical qubit= 2 Distance= 4
Logical qubit= 3 Distance= 4
Logical qubit= 0 Distance= 4
Logical qubit= 1 Distance= 4
Logical qubit= 2 Distance= 4
Logical qubit= 3 Distance= 4
Code parameters: n,k,d= 18 4 4


In [6]:
# the order of two-qubit gates
sX = ['idle', 'idle', 1, 4, 3, 5, 0, 2] ; sZ = ['idle', 3, 5, 0, 1, 2, 4, 'idle'] ;

# Connections of edges in the Tanner graph
lin_order, data_qubits, Xchecks, Zchecks, nbs = get_connection_Tanner(remove_X_list, remove_Z_list, n2)

In [7]:
# SM_cycle

In [8]:
SM_cycle = get_SM_circuit_parallel(remove_X_list, remove_Z_list, lin_order, data_qubits, Xchecks, Zchecks, nbs, sX, sZ ) ;

In [9]:
cycle_append = []
for q in data_qubits:
    cycle_append.append(('final',q))

## Prior probability

In [10]:
# # depolarizing noise model 
from component_error_rates import *

# Setup BP-OSD decoder parameters
my_bp_method = "ms"
my_max_iter = 10000
my_osd_method = "osd_cs"
my_osd_order = 7
my_ms_scaling_factor = 0

## Decoder

In [11]:
Set_decoder_para_18_4_4 = {} ;

In [12]:
max_num_cycles = 6;

In [13]:
for num_cycles in range(1, max_num_cycles + 1):

    # full syndrome measurement circuit
    cycle_repeated = (num_cycles-1) * SM_cycle[2*n2-len(remove_X_list)-len(remove_Z_list):] + \
                SM_cycle[2*n2-len(remove_X_list)-len(remove_Z_list) : -2*n2] + cycle_append  ;
    
    # Generating noisy circuits with a singe faulty operation
    X_circuits, X_Prob = get_Set_noisy_circuits_logical_X(cycle_repeated, error_rate_init, error_rate_idle, error_rate_H, error_rate_cz, \
                                                          error_rate_meas, error_final, error_DD_phaseflip, error_DD_bitflip)
    num_errX=len(X_circuits)
    print('Number of noisy circuits for the logical X state =',num_errX)
    
    
    Z_circuits, Z_Prob = get_Set_noisy_circuits_logical_Z(cycle_repeated, error_rate_init, error_rate_idle, error_rate_H, error_rate_cz, \
                                                          error_rate_meas, error_final, error_DD_phaseflip, error_DD_bitflip)
    num_errZ=len(Z_circuits)
    print('Number of noisy circuits for the logical Z state =',num_errZ)    
    
    channel_probsX, HX, HdecX, HXdict = decoding_X_matrix(X_circuits, X_Prob, num_cycles, SM_cycle, lin_order, n, k, data_qubits, \
                                                          Xchecks, lx, remove_X_list, remove_Z_list) ;
    
    channel_probsZ, HZ, HdecZ, HZdict = decoding_Z_matrix(Z_circuits, Z_Prob, num_cycles, SM_cycle, lin_order, n, k, data_qubits, \
                                                          Zchecks, lz, remove_X_list, remove_Z_list) ;
    
    Set_decoder_para_18_4_4[f"channel_probsX_{num_cycles}"] = channel_probsX ;
    Set_decoder_para_18_4_4[f"HX_{num_cycles}"] = HX ;
    Set_decoder_para_18_4_4[f"HdecX_{num_cycles}"] = HdecX ;

    Set_decoder_para_18_4_4[f"channel_probsZ_{num_cycles}"] = channel_probsZ ;
    Set_decoder_para_18_4_4[f"HZ_{num_cycles}"] = HZ ;
    Set_decoder_para_18_4_4[f"HdecZ_{num_cycles}"] = HdecZ ;  

Number of noisy circuits for the logical X state = 1664
Number of noisy circuits for the logical Z state = 1664
Number of distinct syndrome histories for logical X = 89
Number of distinct syndrome histories for logical Z = 89
Number of noisy circuits for the logical X state = 3328
Number of noisy circuits for the logical Z state = 3328
Number of distinct syndrome histories for logical X = 166
Number of distinct syndrome histories for logical Z = 166
Number of noisy circuits for the logical X state = 4992
Number of noisy circuits for the logical Z state = 4992
Number of distinct syndrome histories for logical X = 243
Number of distinct syndrome histories for logical Z = 243
Number of noisy circuits for the logical X state = 6656
Number of noisy circuits for the logical Z state = 6656
Number of distinct syndrome histories for logical X = 320
Number of distinct syndrome histories for logical Z = 320
Number of noisy circuits for the logical X state = 8320
Number of noisy circuits for the l

In [15]:
Set_decoder_para_18_4_4["hx"] = hx ;
Set_decoder_para_18_4_4["hz"] = hz ;
Set_decoder_para_18_4_4["lx"] = lx ;
Set_decoder_para_18_4_4["lz"] = lz ;

In [16]:
with open('Set_decoder_para_18_4_4.pkl', 'wb') as f:
    pickle.dump(Set_decoder_para_18_4_4, f)

## Classical simulation

In [ ]:
num_cycles = 3;

## import the decoder

In [ ]:
channel_probsX = Set_decoder_para_18_4_4[f"channel_probsX_{num_cycles}"]
HX = Set_decoder_para_18_4_4[f"HX_{num_cycles}"]  ;
HdecX = Set_decoder_para_18_4_4[f"HdecX_{num_cycles}"] ;

channel_probsZ = Set_decoder_para_18_4_4[f"channel_probsZ_{num_cycles}"]
HZ = Set_decoder_para_18_4_4[f"HZ_{num_cycles}"]  ;
HdecZ = Set_decoder_para_18_4_4[f"HdecZ_{num_cycles}"] ;

In [ ]:
bpdX=bposd_decoder(
    HdecX, #the parity check matrix
    channel_probs=channel_probsX, #assign error_rate to each qubit. This will override "error_rate" input variable
    max_iter=my_max_iter, #the maximum number of iterations for BP)
    bp_method=my_bp_method,
    ms_scaling_factor=my_ms_scaling_factor, #min sum scaling factor. If set to zero the variable scaling factor method is used
    osd_method=my_osd_method, #the OSD method. Choose from:  1) "osd_e", "osd_cs", "osd0"
    osd_order=my_osd_order #the osd search depth
    )

bpdZ=bposd_decoder(
    HdecZ, #the parity check matrix
    channel_probs=channel_probsZ, #assign error_rate to each qubit. This will override "error_rate" input variable
    max_iter=my_max_iter, #the maximum number of iterations for BP)
    bp_method=my_bp_method,
    ms_scaling_factor=my_ms_scaling_factor, #min sum scaling factor. If set to zero the variable scaling factor method is used
    osd_method="osd_cs", #the OSD method. Choose from:  1) "osd_e", "osd_cs", "osd0"
    osd_order=my_osd_order #the osd search depth
    )

### Logical X

#### data from classical simulation

In [ ]:
num_trials_x = 5000 ;
syndrome_history_X, final_logical_x_outcome = simulate_syndrome_Lx(num_trials_x, num_cycles, SM_cycle, cycle_append, error_rate_init, \
                error_rate_idle, error_rate_H, error_rate_cz, error_rate_meas, error_final, error_DD_phaseflip, error_DD_bitflip, lx, \
                                                              lin_order, data_qubits, Xchecks, remove_X_list, remove_Z_list)

In [ ]:
assert( len(syndrome_history_X[0]) == (num_cycles+1) * (n2 - len(remove_X_list) ) )
num_detect_x = len(syndrome_history_X[0]) ;

#### Error detections

In [ ]:
labels_x = ['X0', 'X1', 'X3', 'X4', 'X5', 'X6', 'X7']
num_xcheck = len(labels_x) ;

mean_detect_prob_x = np.mean(syndrome_history_X, axis=0) ;
mean_detect_prob_cycle_x = [ np.mean(mean_detect_prob_x[num_xcheck*cycle:num_xcheck*(cycle+1)]) for cycle in range(num_cycles+1) ]

for cycle in range(num_cycles+1) :
    print( f'The mean probability of error detection in cycle {cycle+1} is:', mean_detect_prob_cycle_x[cycle] )

per_x_detect_prob = [ [ mean_detect_prob_x[i + cycle*num_xcheck ] for cycle in range(num_cycles+1) ] for i in range(num_xcheck) ] ;

In [ ]:
plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams['figure.figsize'] = 3, 2
plt.rcParams.update({'font.size': 6})

fig, ax = plt.subplots( dpi = 800)

ax.plot( range(1, num_cycles+2), mean_detect_prob_cycle_x, color = 'purple', linestyle = "-", linewidth=1, 
        marker = 's', markersize = 2, label = "X basis" )

for i in range( num_xcheck ):
    ax.plot( range(1, num_cycles+2), per_x_detect_prob[i], color = 'purple', linestyle = "--", linewidth=0.3, 
        marker = '.', markersize = 1)

ax.legend(frameon=True, loc=(0.7, 0.2), labelspacing=0.4, fontsize='6')
ax.set_xlabel(r'Quantum error correction cycle, $t$', labelpad = 2) 
ax.set_ylabel(f'Error detection probability', labelpad = 3) 
ax.set_ylim(0.0,0.4) 
plt.show()

#### Correlation matrix $p_{ij}$ for error detection events

In [ ]:
def p_ij(x_i, x_j, x_ij):
	return (1 / 2) - (1 / 2) * np.sqrt(  1 - ( 4 * ( x_ij - x_i * x_j)) / (1 - 2 * x_i - 2 * x_j + 4 * x_ij )  )

In [ ]:
X_Set_averages = {
    f"x({s1},{s2})": np.mean([  syndrome_history_X[trial][s1] * syndrome_history_X[trial][s2] for trial in range(num_trials_x)])
    for s1 in range(num_detect_x-1)
    for s2 in range(s1, num_detect_x)  }

X_Set_averages.update({
    f"x({s3})": np.mean([syndrome_history_X[trial][s3] for trial in range(num_trials_x)])
    for s3 in range(num_detect_x)   })

In [ ]:
X_Correlation_matrix = np.zeros( (num_detect_x, num_detect_x) ) ;

for i in range(num_detect_x-1):
    for j in range(i+1, num_detect_x):
        X_Correlation_matrix[i,j] = p_ij(X_Set_averages[f"x({i})"], X_Set_averages[f"x({j})"], X_Set_averages[f"x({i},{j})"])
        X_Correlation_matrix[j,i] = X_Correlation_matrix[i,j] ;

In [ ]:
# [np.max(X_Correlation_matrix[i]  ) for i in range(num_cycles*7+7)]

In [ ]:
# print(tabulate(X_Correlation_matrix))

In [ ]:
plt.figure(figsize=(12, 9))
cax = plt.imshow(X_Correlation_matrix, cmap='YlOrRd', interpolation='nearest', vmin=0.0, vmax=0.2)
plt.colorbar(cax)

plt.gca().invert_yaxis() 
plt.title('Correlation matrix $p_{ij}$', fontsize=16)
plt.xlabel('X Check index i', fontsize=12)
plt.xticks(np.arange(0, num_cycles*7+6, 7)) 
plt.ylabel('X Check index j', fontsize=12)
plt.yticks(np.arange(0, num_cycles*7+6, 7))
# plt.grid(True, which='both', axis='both', linestyle='-', color='gray', linewidth=0.5)

#### error correction

In [ ]:
# correct errors for logical X (X checks)
X_no_correction_good_trials, X_no_correction_good_list, X_good_trials, X_good_list =     \
            error_decoding(num_trials_x, k, bpdX, HX, syndrome_history_X, final_logical_x_outcome )

In [ ]:
print("no error correction for logical X basis:")
print(f'Logical error over {num_cycles+1} cycles (four logical qubits):', 1-X_no_correction_good_trials/num_trials_x)
print(f'Logical error over {num_cycles+1} cycles (single logical qubit):', 1-X_no_correction_good_list/num_trials_x)
print("\n")
print("error correction for logical X basis:")
print(f'Logical error over {num_cycles+1} cycles (four logical qubits):', 1-X_good_trials/num_trials_x)
print(f'Logical error over {num_cycles+1} cycles (single logical qubit):', 1-X_good_list/num_trials_x)

In [ ]:
# # break-even point
# print("logical X point:", 1 - ( (1+np.exp(-cycle_time/T2))/2 )**k)
# print("logical Z point:", 1 - ( (1+np.exp(-cycle_time/T1))/2 )**k)

### Logical Z

In [ ]:
num_trials_z = 5000 ;
syndrome_history_Z, final_logical_z_outcome = simulate_syndrome_Lz(num_trials_z, num_cycles, SM_cycle, cycle_append, error_rate_init, \
                error_rate_idle, error_rate_H, error_rate_cz, error_rate_meas, error_final, error_DD_phaseflip, error_DD_bitflip, lz, \
                                                        lin_order, data_qubits, Zchecks, remove_X_list, remove_Z_list)

In [ ]:
assert( len(syndrome_history_Z[0]) == (num_cycles+1) * (n2 - len(remove_Z_list) ) )
num_detect_z = len(syndrome_history_Z[0]) ;

#### error detections

In [ ]:
labels_z = ['Z0', 'Z1', 'Z2', 'Z5', 'Z6', 'Z7', 'Z8']
num_zcheck = len(labels_z) ;

mean_detect_prob_z = np.mean(syndrome_history_Z, axis=0) ;
mean_detect_prob_cycle_z = [ np.mean(mean_detect_prob_z[num_zcheck*cycle:num_zcheck*(cycle+1)]) for cycle in range(num_cycles+1) ]

for cycle in range(num_cycles+1) :
    print( f'The mean probability of error detection in cycle {cycle+1} is:', mean_detect_prob_cycle_z[cycle] )
    
per_z_detect_prob = [ [ mean_detect_prob_z[i + cycle*num_zcheck ] for cycle in range(num_cycles+1) ] for i in range(num_zcheck) ] ;

In [ ]:
plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams['figure.figsize'] = 3, 2
plt.rcParams.update({'font.size': 6})

fig, ax = plt.subplots( dpi = 800)

ax.plot( range(1, num_cycles+2), mean_detect_prob_cycle_z, color = 'green', linestyle = "-", linewidth=1, 
        marker = 's', markersize = 2, label = "Z basis" )

for i in range( num_zcheck ):
    ax.plot( range(1, num_cycles+2), per_z_detect_prob[i], color = 'green', linestyle = "--", linewidth=0.3, 
        marker = '.', markersize = 1)

ax.legend(frameon=True, loc=(0.7, 0.2), labelspacing=0.4, fontsize='6')
ax.set_xlabel(r'Quantum error correction cycle, $t$', labelpad = 2) 
ax.set_ylabel(f'Error detection probability', labelpad = 3) 
ax.set_ylim(0.1,0.4) 
plt.show()

#### Correlation matrix $p_{ij}$ for error detection events

In [ ]:
Z_Set_averages = {
    f"z({s1},{s2})": np.mean([np.prod([syndrome_history_Z[trial][s1], syndrome_history_Z[trial][s2]]) for trial in range(num_trials_z)])
    for s1 in range(num_detect_z-1)
    for s2 in range(s1, num_detect_z)  }

Z_Set_averages.update({
    f"z({s3})": np.mean([syndrome_history_Z[trial][s3] for trial in range(num_trials_z)])
    for s3 in range(num_detect_z)   })

In [ ]:
Z_Correlation_matrix = np.zeros( (num_detect_z, num_detect_z) ) ;

for i in range(num_detect_z-1):
    for j in range(i+1, num_detect_z):
        Z_Correlation_matrix[i,j] = p_ij(Z_Set_averages[f"z({i})"], Z_Set_averages[f"z({j})"], Z_Set_averages[f"z({i},{j})"])
        Z_Correlation_matrix[j,i] = Z_Correlation_matrix[i,j] ;

In [ ]:
plt.figure(figsize=(8, 6))
cax = plt.imshow(Z_Correlation_matrix, cmap='YlOrRd', interpolation='nearest', vmin=0.0, vmax=0.2)
plt.colorbar(cax)
plt.gca().invert_yaxis() 
plt.title('Correlation matrix $p_{ij}$', fontsize=16)
plt.xlabel('Z Check index i', fontsize=12)
plt.xticks(np.arange(0, num_cycles*7+6, 7)) 
plt.ylabel('Z Check index j', fontsize=12)
plt.grid(True, which='both', axis='both', linestyle='-', color='gray', linewidth=0.5)
plt.yticks(np.arange(0, num_cycles*7+6, 7)) 

#### error correction

In [ ]:
# correct errors for logical Z (Z checks)
Z_no_correction_good_trials, Z_no_correction_good_list, Z_good_trials, Z_good_list =     \
            error_decoding(num_trials_z, k, bpdZ, HZ, syndrome_history_Z, final_logical_z_outcome )

In [ ]:
print("no error correction:")
print(f'Logical error over {num_cycles+1} cycles (four logical qubits):', 1-Z_no_correction_good_trials/num_trials_z)
print(f'Logical error over {num_cycles+1} cycles (single logical qubit):', 1-Z_no_correction_good_list/num_trials_z)
print("\n")
print("error correction:")
print(f'Logical error over {num_cycles+1} cycles (four logical qubits):', 1-Z_good_trials/num_trials_z)
print(f'Logical error over {num_cycles+1} cycles (single logical qubit):', 1-Z_good_list/num_trials_z)

In [ ]:
print( "Estimated logical error per cycle:", 1 - (1-0.704) ** (1/(num_cycles+1)) )